In [ ]:
import numpy as np
from itertools import permutations, combinations

In [ ]:
def make_rotations():
    rotations = []
    for x in np.concatenate((np.eye(3), -np.eye(3))):
        for i in range(3):
            if x[i] == 0:
                y = np.zeros(3)
                y[i] = 1
                z = np.cross(x, y)
                rotations.append(np.array([x, y, z]))
                
                y[i] = -1
                z = np.cross(x, y)
                rotations.append(np.array([x, y, z]))
                
    return np.array(rotations, dtype=int) 

In [ ]:
def common_beacons(scanner1, scanner2):
    count = 0
    for b in scanner1:
        if np.any(np.all(b == scanner2, axis=1)):
            count += 1
    return count

In [ ]:
def find_rotation(scanner1, scanner2, rotations):
    for rotation in rotations:
        rot_scanner = scanner2.dot(rotation)
        for beacon1 in scanner1:
            common = 0
            s1_beacon = scanner1 - beacon1
            for beacon2 in rot_scanner:
                s2_beacon = rot_scanner - beacon2
                common = common_beacons(s1_beacon, s2_beacon)
                if common >= 12:
                    return rotation, beacon1 - beacon2
    return 0, 0

In [ ]:
scanners = {}

with open("input", "r") as f:
    while line := f.readline():
        if line == "\n":
            scanners[scanner_name] = np.array(temp_scanner)
            continue
        if line[:3] == '---':
            scanner_name = line.strip().split()[2]
            temp_scanner = []
            continue
        temp_scanner.append([int(num) for num in line.strip().split(',')])
    scanners[scanner_name] = np.array(temp_scanner)

In [ ]:
rotations = make_rotations()
processed = {"0": scanners["0"]}
scanner_coords = {"0": np.zeros(3)}

### Part 1

In [ ]:
while len(processed.keys()) < len(scanners.keys()):
    for scanner1, scanner2 in permutations(scanners, 2):
        if scanner2 in processed or scanner1 not in processed:
            continue    

        rot, coords = find_rotation(processed[scanner1],
                                    scanners[scanner2],
                                    rotations)
        if not hasattr(rot, "__iter__"):
            continue

        scanner_coords[scanner2] = coords
        processed[scanner2] = scanners[scanner2].dot(rot) + coords   

In [ ]:
processed_tuples = list(map(tuple, np.vstack(list(processed.values()))))

In [ ]:
len(set(processed_tuples))

### Part 2

In [ ]:
def manhattan(point1, point2):
    return sum(abs(x1 - x2) for x1, x2 in zip(point1, point2))

In [ ]:
max(manhattan(point1, point2) for point1, point2 in combinations(scanner_coords.values(), 2))